In [7]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1")
html = BeautifulSoup(url.text)

In [13]:
# 표 전체를 가져오고 싶다 
def make_table(html):
    table = html.find("table", class_ = "type_2")
    table = pd.read_html(str(table))[0]            # 리스트의 0번쨰 인덱스에 표가 들어있음  =  리스트의 0번째만 가져오면 데이터프레임형태로 표 가져올 수 있음 
    del table["N"]
    del table["토론실"]
    table = table.dropna(subset = "종목명")        # 종목명에 누락값이 있는 행은 다 삭제하겠다
    return table

In [18]:
# 맨뒤페이지 가져오고 싶다 
kospi_page = int(html.find("td", class_ = "pgRR").find("a")["href"].split("=")[-1])
kospi_page
                                                # a의 href라는 속성값에 접근해라

47

In [33]:
# 퀴즈: 모든 코스피 페이지를 가져와서 하나의 데이터프레임 만들기
from tqdm import tqdm

total = []
for i in tqdm(range(1,kospi_page+1)):                                          
    url = requests.get(f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={i}")
    html = BeautifulSoup(url.text)
    table = make_table(html)
    total.append(table)
    time.sleep(1 + random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [01:25<00:00,  1.82s/it]


In [32]:
import random
random.random()        # 0~1 사이의 랜덤값 반환 

0.0077024333866705375

In [36]:
total[0].head()        # total의 인덱스 0에서  1페이지의 정보 확인 가능

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE
1,DB하이텍,47350.0,"상승 1,500",+3.27%,5000.0,21023.0,44399.0,25.51,182877.0,9.00,15.65
2,TIGER 미국배당다우존스,12885.0,상승 15,+0.12%,0.0,20996.0,162950.0,0.00,369163.0,NaN,NaN
3,제일기획,18060.0,상승 10,+0.06%,200.0,20776.0,115041.0,22.51,46869.0,11.13,14.99
4,산일전기,67900.0,상승 500,+0.74%,500.0,20672.0,30445.0,8.64,170959.0,7.71,47.00
5,영원무역,46250.0,"상승 1,150",+2.55%,500.0,20494.0,44311.0,29.16,17350.0,4.86,17.03


In [39]:
kospi = pd.concat(total, ignore_index = True)
kospi["소속"]  = "KOSPI"
kospi.to_excel("kospi.xlsx", index=False)
kospi

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,삼성전자,57600.0,상승 300,+0.52%,100.0,3438595.0,5969783.0,50.03,6403270.0,11.64,9.03,KOSPI
1,SK하이닉스,199700.0,"하락 5,300",-2.59%,5000.0,1453821.0,728002.0,55.95,1404404.0,13.94,-15.61,KOSPI
2,LG에너지솔루션,377500.0,"하락 8,000",-2.08%,500.0,883350.0,234000.0,4.54,64569.0,-243.55,6.36,KOSPI
3,삼성바이오로직스,1198000.0,"상승 47,000",+4.08%,2500.0,852665.0,71174.0,13.54,71143.0,81.01,9.12,KOSPI
4,현대차,203000.0,"하락 4,000",-1.93%,5000.0,425115.0,209416.0,37.41,239252.0,4.44,13.68,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...
2305,TIGER 200 산업재,8275.0,하락 5,-0.06%,0.0,20.0,240.0,0.00,10785.0,NaN,NaN,KOSPI
2306,PLUS 코스닥150선물인버스,5465.0,상승 20,+0.37%,0.0,19.0,350.0,0.00,10368.0,NaN,NaN,KOSPI
2307,HANARO 미국애그테크,11695.0,상승 5,+0.04%,0.0,18.0,150.0,0.00,501.0,NaN,NaN,KOSPI
2308,HANARO 글로벌백신치료제MSCI,11475.0,하락 325,-2.75%,0.0,17.0,150.0,0.00,265.0,NaN,NaN,KOSPI


In [ ]:
# 퀴즈: 코스닥까지 모두 가져와서 코스피 + 코스탁을 합친 하나의 데이터프레임 만들기
# PER값이 0보다 크고 10보다 작은 행만 추출해서 저평가 종목 발굴하기


In [41]:
url2 = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1")
html2 = BeautifulSoup(url2.text)

In [42]:
# 맨뒤페이지 가져오고 싶다 
kosdaq_page = int(html2.find("td", class_ = "pgRR").find("a")["href"].split("=")[-1])
kosdaq_page 

36

In [43]:
from tqdm import tqdm

total2 = []
for i in tqdm(range(1,kosdaq_page+1)):                                          
    url2 = requests.get(f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={i}")
    html2 = BeautifulSoup(url2.text)
    table2 = make_table(html2)
    total2.append(table2)
    time.sleep(1 + random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [01:05<00:00,  1.81s/it]


In [44]:
kosdaq = pd.concat(total2, ignore_index = True)
kosdaq["소속"]  = "KOSDAQ"
kosdaq.to_excel("kosdaq.xlsx", index=False)
kosdaq

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,알테오젠,393500.0,"상승 10,500",+2.74%,500.0,209810.0,53319.0,15.82,197826.0,-7154.55,-2.27,KOSDAQ
1,에코프로비엠,135300.0,"하락 5,500",-3.91%,500.0,132325.0,97801.0,10.67,390582.0,-60.89,-0.64,KOSDAQ
2,HLB,88400.0,상승 400,+0.45%,500.0,116146.0,131387.0,22.57,662352.0,-62.08,-32.55,KOSDAQ
3,에코프로,66300.0,"하락 2,300",-3.35%,100.0,88271.0,133138.0,20.02,599991.0,-48.57,3.43,KOSDAQ
4,레인보우로보틱스,390500.0,상승 500,+0.13%,500.0,75756.0,19400.0,8.28,188399.0,-370.14,-0.93,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...
1786,투비소프트,630.0,보합0,0.00%,500.0,62.0,9900.0,2.26,426536.0,-0.20,-41.73,KOSDAQ
1787,셀리버리,152.0,"하락 6,528",-97.72%,500.0,57.0,37183.0,2.30,16946754.0,-0.92,472.60,KOSDAQ
1788,CNH,109.0,보합0,0.00%,2500.0,41.0,37200.0,6.07,0.0,-0.04,-18.80,KOSDAQ
1789,소프트센우,7050.0,상승 140,+2.03%,200.0,25.0,357.0,0.02,107.0,-110.16,NaN,KOSDAQ


In [46]:
stock = pd.concat([kospi, kosdaq], ignore_index = True)
stock.to_excel("stock.xlsx", index=False)
stock

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속
0,삼성전자,57600.0,상승 300,+0.52%,100.0,3438595.0,5969783.0,50.03,6403270.0,11.64,9.03,KOSPI
1,SK하이닉스,199700.0,"하락 5,300",-2.59%,5000.0,1453821.0,728002.0,55.95,1404404.0,13.94,-15.61,KOSPI
2,LG에너지솔루션,377500.0,"하락 8,000",-2.08%,500.0,883350.0,234000.0,4.54,64569.0,-243.55,6.36,KOSPI
3,삼성바이오로직스,1198000.0,"상승 47,000",+4.08%,2500.0,852665.0,71174.0,13.54,71143.0,81.01,9.12,KOSPI
4,현대차,203000.0,"하락 4,000",-1.93%,5000.0,425115.0,209416.0,37.41,239252.0,4.44,13.68,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...
4096,투비소프트,630.0,보합0,0.00%,500.0,62.0,9900.0,2.26,426536.0,-0.20,-41.73,KOSDAQ
4097,셀리버리,152.0,"하락 6,528",-97.72%,500.0,57.0,37183.0,2.30,16946754.0,-0.92,472.60,KOSDAQ
4098,CNH,109.0,보합0,0.00%,2500.0,41.0,37200.0,6.07,0.0,-0.04,-18.80,KOSDAQ
4099,소프트센우,7050.0,상승 140,+2.03%,200.0,25.0,357.0,0.02,107.0,-110.16,NaN,KOSDAQ


In [55]:
stock["저평가"] = (stock['PER'] > 0) & (stock['PER'] < 10)
stock

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,소속,저평가
0,삼성전자,57600.0,상승 300,+0.52%,100.0,3438595.0,5969783.0,50.03,6403270.0,11.64,9.03,KOSPI,False
1,SK하이닉스,199700.0,"하락 5,300",-2.59%,5000.0,1453821.0,728002.0,55.95,1404404.0,13.94,-15.61,KOSPI,False
2,LG에너지솔루션,377500.0,"하락 8,000",-2.08%,500.0,883350.0,234000.0,4.54,64569.0,-243.55,6.36,KOSPI,False
3,삼성바이오로직스,1198000.0,"상승 47,000",+4.08%,2500.0,852665.0,71174.0,13.54,71143.0,81.01,9.12,KOSPI,False
4,현대차,203000.0,"하락 4,000",-1.93%,5000.0,425115.0,209416.0,37.41,239252.0,4.44,13.68,KOSPI,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,투비소프트,630.0,보합0,0.00%,500.0,62.0,9900.0,2.26,426536.0,-0.20,-41.73,KOSDAQ,False
4097,셀리버리,152.0,"하락 6,528",-97.72%,500.0,57.0,37183.0,2.30,16946754.0,-0.92,472.60,KOSDAQ,False
4098,CNH,109.0,보합0,0.00%,2500.0,41.0,37200.0,6.07,0.0,-0.04,-18.80,KOSDAQ,False
4099,소프트센우,7050.0,상승 140,+2.03%,200.0,25.0,357.0,0.02,107.0,-110.16,NaN,KOSDAQ,False


In [65]:
# 요청하는 서버를 파이썬이 아닌 아이폰이라고 속일거임 
dic = {"User-Agent" : "mozilla/5.0 (macintosh; intel mac os x 10_15_7"}

url = requests.get("https://finance.naver.com/item/sise_day.naver?code=005930", headers = dic)
html = BeautifulSoup(url.text)
html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버페이 증권</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250220171615/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250220171615/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250220171615/css/finance_header.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250220171615/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250220171615/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250220171615/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20250220171615/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ss

In [68]:
table = html.find("table")
table = pd.read_html(str(table))[0].dropna()
table

,날짜,종가,전일비,시가,고가,저가,거래량
1,2025.02.25,57400.0,상승 100,56600.0,57800.0,56500.0,8285317.0
2,2025.02.24,57300.0,하락 900,57300.0,57700.0,57200.0,14138471.0
3,2025.02.21,58200.0,하락 200,58400.0,58500.0,57100.0,22198428.0
4,2025.02.20,58400.0,하락 300,59100.0,59100.0,58100.0,21589059.0
5,2025.02.19,58700.0,"상승 1,800",57400.0,58900.0,57300.0,25395151.0
9,2025.02.18,56900.0,상승 900,56200.0,57200.0,55900.0,22131007.0
10,2025.02.17,56000.0,보합0,56100.0,56500.0,55700.0,11916027.0
11,2025.02.14,56000.0,상승 200,56000.0,57300.0,56000.0,23979779.0
12,2025.02.13,55800.0,보합0,56100.0,56400.0,55600.0,22448377.0
13,2025.02.12,55800.0,상승 100,55100.0,55900.0,54500.0,26428596.0


In [103]:
url = requests.get("https://finance.naver.com/sise/dividend_list.naver?&page=1")
html = BeautifulSoup(url.text)
#html

In [104]:
import numpy as np

# 전체 페이지 불러와서, 데이터 전처리하기
# 3년전 < 2년전 < 1년전 

table = html.find("table", class_ = "type_1 tb_ty")
table = pd.read_html(str(table), header = 1)[0]          # 1번째행을 헤더로 해서 실행하겠다 
table = table.dropna(subset = "종목명")


In [105]:
page = int(html.find("td", class_ = "pgRR").find("a")["href"].split("=")[-1])
page

17

In [106]:
def make_table(html):
    table = html.find("table", class_ = "type_1 tb_ty")
    table = pd.read_html(str(table), header = 1)[0]          # 1번째행을 헤더로 해서 실행하겠다 
    table = table.dropna(subset = "종목명")
    return table

In [100]:
from tqdm import tqdm

total = []
for i in tqdm(range(1,page+1)):                                          
    url = requests.get(f"https://finance.naver.com/sise/dividend_list.naver?&page={i}")
    html = BeautifulSoup(url.text)
    table = make_table(html)
    total.append(table)
    time.sleep(1 + random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:28<00:00,  1.70s/it]


In [107]:
table = pd.concat(total, ignore_index = True)
table

,종목명,현재가,기준월,배당금,수익률 (%),배당성향 (%),ROE (%),PER (배),PBR (배),1년전,2년전,3년전
0,이지스밸류리츠,4375.0,24.12,823.0,18.81,-1361.40,-0.57,-116.13,0.66,406.0,293,292
1,제이알글로벌리츠,2520.0,24.06,390.0,15.48,158.83,2.82,32.42,0.94,380.0,366,330
2,레드캡투어,14050.0,25.03,2150.0,15.30,177.53,-,-,-,450.0,450,400
3,NH프라임리츠,4545.0,24.11,658.0,14.48,137.10,6.38,11.26,0.72,531.0,225,248
4,신한알파리츠,6050.0,24.09,806.0,13.32,547.91,0.61,183.31,1.19,365.0,367,321
...,...,...,...,...,...,...,...,...,...,...,...,...
840,포스코엠텍,13970.0,25.03,20.0,0.14,146.62,-,-,-,35.0,37.0,80
841,삼천당제약,185500.0,24.12,200.0,0.11,-,-,-,-,0.0,0.0,0
842,솔루스첨단소재,10160.0,24.12,10.0,0.10,-,-,-,-,50.0,50.0,48
843,대주전자재료,115400.0,24.12,100.0,0.09,-,-,-,-,0.0,100.0,100


In [108]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   종목명       845 non-null    object 
 1   현재가       845 non-null    float64
 2   기준월       845 non-null    object 
 3   배당금       845 non-null    float64
 4   수익률 (%)   845 non-null    float64
 5   배당성향 (%)  845 non-null    object 
 6   ROE (%)   845 non-null    object 
 7   PER (배)   845 non-null    object 
 8   PBR (배)   845 non-null    object 
 9   1년전       845 non-null    float64
 10  2년전       845 non-null    object 
 11  3년전       845 non-null    object 
dtypes: float64(4), object(8)
memory usage: 79.3+ KB


In [123]:
table["3년전"] = table["3년전"].replace("-", 0).astype("float")
table["2년전"] = table["2년전"].replace("-", 0).astype("float")
table["1년전"] = table["1년전"].replace("-", 0).astype("float")
table["배당금"] = table["배당금"].replace("-", 0).astype("float")
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   종목명       845 non-null    object 
 1   현재가       845 non-null    float64
 2   기준월       845 non-null    object 
 3   배당금       845 non-null    float64
 4   수익률 (%)   845 non-null    float64
 5   배당성향 (%)  845 non-null    object 
 6   ROE (%)   845 non-null    object 
 7   PER (배)   845 non-null    object 
 8   PBR (배)   845 non-null    object 
 9   1년전       845 non-null    float64
 10  2년전       845 non-null    float64
 11  3년전       845 non-null    float64
dtypes: float64(6), object(6)
memory usage: 79.3+ KB


In [126]:
A = table["3년전"] < table["2년전"]
B = table["2년전"] < table["1년전"]
C = table["1년전"] < table["배당금"]
df2 = table[A & B & C]
df2

,종목명,현재가,기준월,배당금,수익률 (%),배당성향 (%),ROE (%),PER (배),PBR (배),1년전,2년전,3년전
0,이지스밸류리츠,4375.0,24.12,823.0,18.81,-1361.40,-0.57,-116.13,0.66,406.0,293.0,292.0
1,제이알글로벌리츠,2520.0,24.06,390.0,15.48,158.83,2.82,32.42,0.94,380.0,366.0,330.0
17,현대해상,24150.0,24.03,2063.0,8.54,26.61,7.81,4.56,0.40,1965.0,1480.0,1000.0
24,도이치모터스,4735.0,24.12,380.0,8.02,-,-,-,-,370.0,360.0,350.0
32,HS애드,7200.0,25.03,550.0,7.64,39.29,-,-,-,450.0,400.0,350.0
...,...,...,...,...,...,...,...,...,...,...,...,...
784,한화에어로스페이스,677000.0,24.12,3500.0,0.52,6.94,-,-,-,1800.0,1000.0,700.0
789,포스코DX,25000.0,25.03,125.0,0.50,-,-,-,-,100.0,75.0,50.0
805,유한양행,122100.0,24.12,500.0,0.41,-,-,-,-,430.0,365.0,349.0
813,삼양식품,869000.0,25.02,3300.0,0.38,-,-,-,-,2100.0,1400.0,1000.0


In [132]:
from datetime import datetime
today = str(datetime.today().date())

text = f"{today}기준 3년 연속 배당금이 상승한 기업 리스트\n\n"

for i in range(len(df2)):
    row = df2.iloc[i]
    name = row['종목명']
    a,b,c,d = row['배당금'], row['1년전'], row['2년전'], row['3년전']
    text += f'{name} : {int(d):,}원 → {int(c):,}원 → {int(b):,}원 → {int(a):,}원\n'

f = open("3년 연속 배당금 상승 기업.txt", 'wt')
f.write(text)
f.close()

In [131]:
from datetime import datetime
today = str(datetime.today().date())
today

'2025-02-25'

In [122]:
cnt = 0
for i in range(0, len(table)):
    if (table["3년전"][i] < table["2년전"][i] < table["1년전"][i] < table["배당금"][i]):
        cnt += 1
cnt

81